In [1]:
from gym import Env
import numpy as np
from gym.spaces import MultiDiscrete,Box
from graph_tool.all import *
from makegraph import *

In [2]:
def simulatepandemic(self,actions):
    action = np.divide(actions,np.sum(actions)) #relative availability of vaccine is calculated
    update_state(self.g,action)
    self.state = graph_to_matrix(self.g)
    self.timestep += 1
    return self.state

def initializepandemic(self):
    self.g = make_graph(self.size,self.distribution)
    self.state = graph_to_matrix(self.g)
    return self.state

In [3]:
class PanEnv(Env):
    def __init__(self,size,distribution):
        self.size = size #population size
        
        self.action_space = MultiDiscrete(nvec=[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10])
        self.observation_space = Box(low=-np.inf,high=np.inf,shape=[self.size,6])
        
        
        self.distribution = distribution #age distribution, country name as string
        self.g = make_graph(self.size,self.distribution)
                
        #state observation as matrix
        self.state = graph_to_matrix(self.g)
        
        self.timestep = 1
    def step(self,actions):
        state = simulatepandemic(self,actions)
        #observation (object): agent's observation of the current environment
        reward = -np.sum(state[:,5])
        #reward (float) : amount of reward returned after previous action
        #negative reward: punishment > change weights a lot, push away from causing weights, positive rewards pull
        #do reward compared to reward from previous step
        #naive example: reward = -sum(infected) > we want a reward where the cumulative sum of infections until end
        #is minimized
        #exp. solution: store information in self, summed infections, normalized by time
        done = False
        if((self.timestep > 19) or np.sum(self.state[:,1])==0):
            done = True
        #done (bool): episode is done after 20 timesteps or when there are no longer infected agents
        info = {}
        #info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)'''
        return state, reward, done, info
    def reset(self):
        #returns initial state
        self.state = initializepandemic(self)
        return self.state

In [4]:
from stable_baselines3.common.env_checker import check_env
#check_env(Env)

In [5]:
#matrix representation:
#[S, I, R, Sv, Iv, D]

In [6]:
env = PanEnv(size=1000,distribution='Japan')
env.reset()

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [7]:
actions = np.full(20,1)

In [8]:
env.step(actions)

(array([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        ...,
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]]),
 -1.0,
 True,
 {})

In [9]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import BaseCallback
#tensorflow for graphs
import tensorflow as tf
# Parallel environments

env = DummyVecEnv([lambda: PanEnv(size=1000,distribution='Japan')])

2021-11-27 22:07:23.573898: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 22:07:23.573919: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
#the class is needed if we want to sample more frequently
class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """

    def __init__(self, verbose=1):
        super(TensorboardCallback, self).__init__(verbose)
        

    def _on_step(self) -> bool:
        # Log scalar value (here a random variable)
        value = np.random.random()
        #self.logger.record('random_value', value)
        #self.logger.dump(self.num_timesteps)
        return True

In [11]:
model = PPO("MlpPolicy", env, verbose=1,tensorboard_log="./ppo_name_tensorboard/") #multilayer
model.learn(total_timesteps=25000,tb_log_name="first_run",callback=TensorboardCallback()) #training loop
#model.save("ppo_cartpole")

#del model # remove to demonstrate saving and loading

#model = PPO.load("ppo_cartpole")
#https://stable-baselines.readthedocs.io/en/master/guide/tensorboard.html?highlight=tensorboard
#^^^ tensorboard for displaying things, but I am stuck, upd: i guess it works now ^^^

Using cpu device
Logging to ./ppo_name_tensorboard/first_run_14
-----------------------------
| time/              |      |
|    fps             | 3    |
|    iterations      | 1    |
|    time_elapsed    | 524  |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 4           |
|    iterations           | 2           |
|    time_elapsed         | 866         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.025057014 |
|    clip_fraction        | 0.314       |
|    clip_range           | 0.2         |
|    entropy_loss         | -46         |
|    explained_variance   | 0.000381    |
|    learning_rate        | 0.0003      |
|    loss                 | 1.49e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00449    |
|    value_loss           | 2.01e+05    |
------------

KeyboardInterrupt: 

In [ ]:
model.save("ppo_1")

In [ ]:
gym.spaces??

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
log_dir = "/gym/"

env = Monitor(env, log_dir)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
obs = env.reset()
dones = False
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

In [ ]:
#split into learning and testing
model.learn(total_timesteps = 5000)
# store/accumulate rewards